In [1]:
import json
import pandas as pd
import numpy as np

# Read Json

In [2]:
with open("../../events/events_England.json", "r") as file:
    event_data_2 = json.load(file)

df2 = pd.DataFrame(event_data_2)

with open("../../events/events_France.json", "r") as file:
    event_data_3 = json.load(file)

df3 = pd.DataFrame(event_data_3)

with open("../../events/events_Germany.json", "r") as file:
    event_data_4 = json.load(file)

df4 = pd.DataFrame(event_data_4)

with open("../../events/events_Italy.json", "r") as file:
    event_data_5 = json.load(file)

df5 = pd.DataFrame(event_data_5)

with open("../../events/events_Spain.json", "r") as file:
    event_data_6 = json.load(file)

df6 = pd.DataFrame(event_data_6)

with open("../../events/teams.json", "r") as file:
    teams_data = json.load(file)

teams_df = pd.DataFrame(teams_data)

In [3]:
all_events_df = pd.concat([df2, df3, df4, df5, df6], ignore_index=True)

In [11]:
passes_df = all_events_df[all_events_df['eventName'] == 'Pass']
in_possession_df = all_events_df[all_events_df['eventName'].isin(['Shot','Pass'])]

In [6]:
# Determine the previous team ID in the same match
passes_df['previous_teamId'] = passes_df.groupby('matchId')['teamId'].shift(1)

# Identify transitions where the team ID changes and it's within the same match
transitions = passes_df[(passes_df['teamId'] != passes_df['previous_teamId']) & (passes_df['previous_teamId'].notna())]

# Create a transitions DataFrame with required columns for merging
transitions['endSec'] = transitions['eventSec'] + 10
transition_times = transitions[['id', 'matchId', 'eventSec', 'endSec']]

<ipython-input-6-6110caba3bd7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df['previous_teamId'] = passes_df.groupby('matchId')['teamId'].shift(1)
<ipython-input-6-6110caba3bd7>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transitions['endSec'] = transitions['eventSec'] + 10


In [7]:
# Convert data types to more memory efficient types
transitions['matchId'] = transitions['matchId'].astype('int32')
transitions['eventSec'] = transitions['eventSec'].astype('float32')
transitions['endSec'] = transitions['endSec'].astype('float32')

# Check if converting teamId to a smaller integer type is possible
transitions['teamId'] = pd.to_numeric(transitions['teamId'], downcast='integer')

<ipython-input-7-5f7b6da8d4f6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transitions['matchId'] = transitions['matchId'].astype('int32')
<ipython-input-7-5f7b6da8d4f6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transitions['eventSec'] = transitions['eventSec'].astype('float32')
<ipython-input-7-5f7b6da8d4f6>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

In [12]:
passes_df['previous_teamId'] = passes_df.groupby('matchId')['teamId'].shift(1)
transitions = passes_df[(passes_df['teamId'] != passes_df['previous_teamId']) & (passes_df['previous_teamId'].notna())]

# Add a 10-second window for the transition end time
transitions['transition_end_time'] = transitions['eventSec'] + 10

# Prepare the minimal necessary data for merging
transitions_minimal = transitions[['id', 'matchId', 'eventSec', 'transition_end_time']]
in_possession_df_minimal = in_possession_df[['id', 'matchId', 'eventSec', 'teamId', "subEventName", "positions", "eventName"]]

<ipython-input-12-df402b5011d2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df['previous_teamId'] = passes_df.groupby('matchId')['teamId'].shift(1)
<ipython-input-12-df402b5011d2>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transitions['transition_end_time'] = transitions['eventSec'] + 10


In [13]:
# Define a function to process chunks
def process_chunk(chunk):
    chunk_merged = chunk.merge(transitions_minimal, on='matchId', suffixes=('', '_trans'))
    return chunk_merged[(chunk_merged['eventSec'] >= chunk_merged['eventSec_trans']) & (chunk_merged['eventSec'] <= chunk_merged['transition_end_time'])]

# Process in chunks
chunk_size = 100000
result_frames = []
for start in range(0, in_possession_df_minimal.shape[0], chunk_size):
    chunk = in_possession_df_minimal.iloc[start:start + chunk_size]
    result_frames.append(process_chunk(chunk))

In [14]:
in_poss_merged_df = pd.concat(result_frames)
in_poss_merged_df = in_poss_merged_df.drop_duplicates(subset='id', keep='first')

In [15]:
in_poss_merged_df

,id,matchId,eventSec,teamId,subEventName,positions,eventName,id_trans,eventSec_trans,transition_end_time
783,177959175,2499719,10.302366,1609,Simple pass,"[{'y': 95, 'x': 41}, {'y': 88, 'x': 72}]",Pass,177960133,8.449574,18.449574
957,177959177,2499719,12.548934,1609,Simple pass,"[{'y': 88, 'x': 72}, {'y': 75, 'x': 77}]",Pass,177960133,8.449574,18.449574
1044,177959186,2499719,13.961228,1631,Head pass,"[{'y': 25, 'x': 23}, {'y': 15, 'x': 39}]",Pass,177959186,13.961228,23.961228
1218,177959178,2499719,15.320341,1609,Head pass,"[{'y': 80, 'x': 67}, {'y': 61, 'x': 59}]",Pass,177959186,13.961228,23.961228
1392,177959179,2499719,18.051875,1609,Head pass,"[{'y': 61, 'x': 59}, {'y': 45, 'x': 45}]",Pass,177959186,13.961228,23.961228
...,...,...,...,...,...,...,...,...,...,...
935365,253302672,2565927,2813.117759,675,High pass,"[{'y': 27, 'x': 36}, {'y': 90, 'x': 33}]",Pass,253302669,2808.463744,2818.463744
935519,253302675,2565927,2817.686043,675,Simple pass,"[{'y': 90, 'x': 33}, {'y': 94, 'x': 45}]",Pass,253302669,2808.463744,2818.463744
936598,253302650,2565927,2874.769542,682,Simple pass,"[{'y': 16, 'x': 7}, {'y': 6, 'x': 27}]",Pass,253302650,2874.769542,2884.769542
936752,253302651,2565927,2876.959208,682,Simple pass,"[{'y': 6, 'x': 27}, {'y': 4, 'x': 40}]",Pass,253302650,2874.769542,2884.769542


# in_poss_merged_df is ready (update below to create final_df)

# Feature engineering - Pass DF - Pass Motifs

In [6]:
passes_df = all_events_df[all_events_df['eventName'] == 'Pass']

# Concatenating teamId and matchId to create a unique identifier for each team in each match
passes_df['uniqueTeamId'] = passes_df['teamId'].astype(str) + "---" + passes_df['matchId'].astype(str)

<ipython-input-6-846069bd07cf>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df['uniqueTeamId'] = passes_df['teamId'].astype(str) + "---" + passes_df['matchId'].astype(str)


In [7]:
# Function to classify pass motifs
def classify_pass_motif(passes):
    players = passes['playerId'].values
    if len(set(players)) == 2:  # ABAB
        return 'ABAB'
    elif len(set(players)) == 3:
        if players[0] == players[2] or players[1] == players[3]:  # ABCA or ABCB
            return 'ABCA' if players[0] == players[2] else 'ABCB'
        else:
            return 'ABCD'  # ABCD for distinct players or other scenarios
    return 'ABCD'

# Apply the classification within a rolling window
def apply_motif_classification(df):
    # Ensure the DataFrame is sorted by event time
    df = df.sort_values(by='eventSec')
    motifs = []
    # Using a rolling window of 4 events
    for i in range(len(df) - 3):
        window = df.iloc[i:i+4]
        motif = classify_pass_motif(window)
        motifs.append((window.iloc[-1]['uniqueTeamId'], motif))
    return pd.DataFrame(motifs, columns=['uniqueTeamId', 'Motif'])

In [8]:
motif_classification_results = passes_df.groupby('uniqueTeamId').apply(apply_motif_classification)

In [9]:
motif_classification_results.reset_index(drop=True, inplace=True)

In [10]:
motif_counts = motif_classification_results.groupby(['uniqueTeamId', 'Motif']).size().unstack(fill_value=0)

In [11]:
motif_ratios = motif_counts.div(motif_counts.sum(axis=1), axis=0)
# df to be used in clustering

In [12]:
motif_ratios

Motif,ABAB,ABCA,ABCB,ABCD
uniqueTeamId,,,,
10531---2499726,0.039755,0.091743,0.088685,0.779817
10531---2499736,0.045570,0.106329,0.103797,0.744304
10531---2499741,0.022989,0.105747,0.105747,0.765517
10531---2499757,0.027344,0.103516,0.101562,0.767578
10531---2499766,0.032258,0.101382,0.105991,0.760369
...,...,...,...,...
756---2565880,0.029213,0.110112,0.112360,0.748315
756---2565892,0.036842,0.113158,0.105263,0.744737
756---2565904,0.031496,0.089239,0.089239,0.790026


In [13]:
motif_stats = motif_ratios.agg(['mean', 'std']).T

In [14]:
motif_stats

,mean,std
Motif,,
ABAB,0.034688,0.011951
ABCA,0.092979,0.014931
ABCB,0.093592,0.015049
ABCD,0.778740,0.033290


# Feature engineering - Shot DF

In [15]:
shot_df = all_events_df[all_events_df['eventName'] == 'Shot']
shot_df['uniqueTeamId'] = shot_df['teamId'].astype(str) + "---" + shot_df['matchId'].astype(str)

<ipython-input-15-396a5f5d4462>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shot_df['uniqueTeamId'] = shot_df['teamId'].astype(str) + "---" + shot_df['matchId'].astype(str)


In [17]:
import math

def calculate_shot_distance(row):
    if row['positions'] is not None and len(row['positions']) > 0:
        loc = row['positions'][0]
        # Normalize locations
        x = loc['x'] * (120 / 100.0)  # Normalizing the x coordinate
        y = loc['y'] * (80 / 100.0)   # Normalizing the y coordinate
        # Calculate Euclidean distance from (0,0)
        distance = math.sqrt((120-x)**2 + (80-y)**2)
        return distance
    return None

# Apply the function to calculate shot distance and add it to the DataFrame
shot_df['shot_distance'] = shot_df.apply(calculate_shot_distance, axis=1)

<ipython-input-17-103ed8455f10>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shot_df['shot_distance'] = shot_df.apply(calculate_shot_distance, axis=1)


In [18]:
def classify_shot_by_distance(distance):
    if distance <= 38:
        return "Near"
    elif distance <= 52:
        return "Middle Distance"
    else:
        return "Far"

# Assuming shot_df has a 'shot_distance' column calculated as before
shot_df['shot_class'] = shot_df['shot_distance'].apply(classify_shot_by_distance)

# Grouping shots by their class and counting
shot_class_counts = shot_df.groupby('uniqueTeamId')['shot_class'].value_counts().unstack(fill_value=0)
shot_counts = shot_df.groupby('uniqueTeamId')['subEventName'].value_counts().unstack(fill_value=0)

<ipython-input-18-0451b5045683>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shot_df['shot_class'] = shot_df['shot_distance'].apply(classify_shot_by_distance)


In [19]:
shot_df_final = pd.concat([shot_counts, shot_class_counts], axis=1)

In [21]:
shot_df_final

,Shot,Far,Middle Distance,Near
uniqueTeamId,,,,
10531---2499726,4,1,1,2
10531---2499736,7,3,4,0
10531---2499741,6,1,2,3
10531---2499757,8,0,5,3
10531---2499766,3,0,2,1
...,...,...,...,...
756---2565880,19,4,8,7
756---2565892,11,6,3,2
756---2565904,14,0,9,5


In [22]:
shot_df_final.rename(columns = {'Shot':'Total Shots'}, inplace = True) 

shot_df_final["Far"] = shot_df_final["Far"]/shot_df_final["Total Shots"]
shot_df_final["Middle Distance"] = shot_df_final["Middle Distance"]/shot_df_final["Total Shots"]
shot_df_final["Near"] = shot_df_final["Near"]/shot_df_final["Total Shots"]


shot_df_final.rename(columns = {'Near':'Near Shots Ratio', 'Middle Distance':'Middle Distance Shots Ratio', 'Far':'Far Shots Ratio',}, inplace = True) 

In [23]:
shot_df_final

,Total Shots,Far Shots Ratio,Middle Distance Shots Ratio,Near Shots Ratio
uniqueTeamId,,,,
10531---2499726,4,0.250000,0.250000,0.500000
10531---2499736,7,0.428571,0.571429,0.000000
10531---2499741,6,0.166667,0.333333,0.500000
10531---2499757,8,0.000000,0.625000,0.375000
10531---2499766,3,0.000000,0.666667,0.333333
...,...,...,...,...
756---2565880,19,0.210526,0.421053,0.368421
756---2565892,11,0.545455,0.272727,0.181818
756---2565904,14,0.000000,0.642857,0.357143


# Feature engineering - Pass DF - Pass Types

In [25]:
def determine_zone(position):
    if 'x' in position and 'y' in position:
        x, y = position['x'], position['y']
        zone = int(x / 33.333) + 1
        if zone == 4:
            zone = 3
        return zone
    else:
        return None
    
# Apply the function to determine zones for each event
passes_df['Zone'] = passes_df['positions'].apply(lambda pos: determine_zone(pos[0]) if len(pos) > 0 else None)

<ipython-input-25-1dcee42e8a09>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df['Zone'] = passes_df['positions'].apply(lambda pos: determine_zone(pos[0]) if len(pos) > 0 else None)


In [26]:
passes_df

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,uniqueTeamId,Zone
0,8,Simple pass,[{'id': 1801}],25413,"[{'y': 49, 'x': 49}, {'y': 78, 'x': 31}]",2499719,Pass,1609,1H,2.758649,85,177959171,1609---2499719,2
1,8,High pass,[{'id': 1801}],370224,"[{'y': 78, 'x': 31}, {'y': 75, 'x': 51}]",2499719,Pass,1609,1H,4.946850,83,177959172,1609---2499719,1
2,8,Head pass,[{'id': 1801}],3319,"[{'y': 75, 'x': 51}, {'y': 71, 'x': 35}]",2499719,Pass,1609,1H,6.542188,82,177959173,1609---2499719,2
3,8,Head pass,[{'id': 1801}],120339,"[{'y': 71, 'x': 35}, {'y': 95, 'x': 41}]",2499719,Pass,1609,1H,8.143395,82,177959174,1609---2499719,2
4,8,Simple pass,[{'id': 1801}],167145,"[{'y': 95, 'x': 41}, {'y': 88, 'x': 72}]",2499719,Pass,1609,1H,10.302366,85,177959175,1609---2499719,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3071386,8,Hand pass,[{'id': 1801}],3486,"[{'y': 29, 'x': 8}, {'y': 19, 'x': 14}]",2565927,Pass,682,2H,2925.547631,81,253302662,682---2565927,1
3071388,8,Simple pass,[{'id': 1801}],3695,"[{'y': 34, 'x': 36}, {'y': 52, 'x': 45}]",2565927,Pass,682,2H,2931.782904,85,253302665,682---2565927,2
3071390,8,Simple pass,[{'id': 1801}],20623,"[{'y': 25, 'x': 66}, {'y': 2, 'x': 88}]",2565927,Pass,682,2H,2939.077491,85,253302671,682---2565927,2
3071392,8,Cross,"[{'id': 401}, {'id': 1802}]",122832,"[{'y': 21, 'x': 97}, {'y': 26, 'x': 92}]",2565927,Pass,682,2H,2942.098761,80,253302674,682---2565927,3


In [27]:
import math

def calculate_pass_details_from_single_column(row):
    if row['positions'] is not None and len(row['positions']) == 2:
        # Extract and normalize start and end locations
        start_loc = row['positions'][0]
        end_loc = row['positions'][1]
        
        start_x = start_loc['x'] * (120 / 100.0)
        start_y = start_loc['y'] * (80 / 100.0)
        end_x = end_loc['x'] * (120 / 100.0)
        end_y = end_loc['y'] * (80 / 100.0)
        
        # Calculate pass length and angle
        delta_x = end_x - start_x
        delta_y = end_y - start_y
        pass_length = math.sqrt(delta_x**2 + delta_y**2)
        pass_angle = math.atan2(delta_y, delta_x)
        
        return pd.Series([pass_length, pass_angle])
    else:
        return pd.Series([None, None])

In [28]:
passes_df[['calculated_pass_length', 'calculated_pass_angle']] = passes_df.apply(calculate_pass_details_from_single_column, axis=1)


c:\users\eged1\appdata\local\programs\python\python38\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [31]:
def determine_pass_direction(angle):
    # Forward Pass: -π/4 < angle < π/4
    if -np.pi/4 < angle < np.pi/4:
        return "Forward Pass"
    # Backward Pass: 3π/4 < angle <= π or -π <= angle < -3π/4
    elif 3*np.pi/4 < angle <= np.pi or -np.pi <= angle < -3*np.pi/4:
        return "Backward Pass"
    # Side Pass: Otherwise
    else:
        return "Side Pass"

In [32]:
passes_df['pass_direction'] = passes_df['calculated_pass_angle'].apply(determine_pass_direction)

<ipython-input-32-a0b0c4f17a64>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df['pass_direction'] = passes_df['calculated_pass_angle'].apply(determine_pass_direction)


In [33]:
passes_df

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,uniqueTeamId,Zone,calculated_pass_length,calculated_pass_angle,pass_direction
0,8,Simple pass,[{'id': 1801}],25413,"[{'y': 49, 'x': 49}, {'y': 78, 'x': 31}]",2499719,Pass,1609,1H,2.758649,85,177959171,1609---2499719,2,31.698580,2.320495,Side Pass
1,8,High pass,[{'id': 1801}],370224,"[{'y': 78, 'x': 31}, {'y': 75, 'x': 51}]",2499719,Pass,1609,1H,4.946850,83,177959172,1609---2499719,1,24.119701,-0.099669,Forward Pass
2,8,Head pass,[{'id': 1801}],3319,"[{'y': 75, 'x': 51}, {'y': 71, 'x': 35}]",2499719,Pass,1609,1H,6.542188,82,177959173,1609---2499719,2,19.464840,-2.976444,Backward Pass
3,8,Head pass,[{'id': 1801}],120339,"[{'y': 71, 'x': 35}, {'y': 95, 'x': 41}]",2499719,Pass,1609,1H,8.143395,82,177959174,1609---2499719,2,20.505609,1.212026,Side Pass
4,8,Simple pass,[{'id': 1801}],167145,"[{'y': 95, 'x': 41}, {'y': 88, 'x': 72}]",2499719,Pass,1609,1H,10.302366,85,177959175,1609---2499719,2,37.619144,-0.149416,Forward Pass
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3071386,8,Hand pass,[{'id': 1801}],3486,"[{'y': 29, 'x': 8}, {'y': 19, 'x': 14}]",2565927,Pass,682,2H,2925.547631,81,253302662,682---2565927,1,10.762899,-0.837981,Side Pass
3071388,8,Simple pass,[{'id': 1801}],3695,"[{'y': 34, 'x': 36}, {'y': 52, 'x': 45}]",2565927,Pass,682,2H,2931.782904,85,253302665,682---2565927,2,18.000000,0.927295,Side Pass
3071390,8,Simple pass,[{'id': 1801}],20623,"[{'y': 25, 'x': 66}, {'y': 2, 'x': 88}]",2565927,Pass,682,2H,2939.077491,85,253302671,682---2565927,2,32.179497,-0.608689,Forward Pass
3071392,8,Cross,"[{'id': 401}, {'id': 1802}]",122832,"[{'y': 21, 'x': 97}, {'y': 26, 'x': 92}]",2565927,Pass,682,2H,2942.098761,80,253302674,682---2565927,3,7.211103,2.553590,Backward Pass


In [58]:
np.unique(passes_df.subEventName)

array(['Cross', 'Hand pass', 'Head pass', 'High pass', 'Launch',
       'Simple pass', 'Smart pass'], dtype=object)

In [37]:
passes_df

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,uniqueTeamId,calculated_pass_length,calculated_pass_angle
0,8,Simple pass,[{'id': 1801}],25413,"[{'y': 49, 'x': 49}, {'y': 78, 'x': 31}]",2499719,Pass,1609,1H,2.758649,85,177959171,1609---2499719,31.698580,2.320495
1,8,High pass,[{'id': 1801}],370224,"[{'y': 78, 'x': 31}, {'y': 75, 'x': 51}]",2499719,Pass,1609,1H,4.946850,83,177959172,1609---2499719,24.119701,-0.099669
2,8,Head pass,[{'id': 1801}],3319,"[{'y': 75, 'x': 51}, {'y': 71, 'x': 35}]",2499719,Pass,1609,1H,6.542188,82,177959173,1609---2499719,19.464840,-2.976444
3,8,Head pass,[{'id': 1801}],120339,"[{'y': 71, 'x': 35}, {'y': 95, 'x': 41}]",2499719,Pass,1609,1H,8.143395,82,177959174,1609---2499719,20.505609,1.212026
4,8,Simple pass,[{'id': 1801}],167145,"[{'y': 95, 'x': 41}, {'y': 88, 'x': 72}]",2499719,Pass,1609,1H,10.302366,85,177959175,1609---2499719,37.619144,-0.149416
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3071386,8,Hand pass,[{'id': 1801}],3486,"[{'y': 29, 'x': 8}, {'y': 19, 'x': 14}]",2565927,Pass,682,2H,2925.547631,81,253302662,682---2565927,10.762899,-0.837981
3071388,8,Simple pass,[{'id': 1801}],3695,"[{'y': 34, 'x': 36}, {'y': 52, 'x': 45}]",2565927,Pass,682,2H,2931.782904,85,253302665,682---2565927,18.000000,0.927295
3071390,8,Simple pass,[{'id': 1801}],20623,"[{'y': 25, 'x': 66}, {'y': 2, 'x': 88}]",2565927,Pass,682,2H,2939.077491,85,253302671,682---2565927,32.179497,-0.608689
3071392,8,Cross,"[{'id': 401}, {'id': 1802}]",122832,"[{'y': 21, 'x': 97}, {'y': 26, 'x': 92}]",2565927,Pass,682,2H,2942.098761,80,253302674,682---2565927,7.211103,2.553590


In [28]:
#passes_count = passes_df.groupby('teamId')['subEventName'].value_counts().unstack(fill_value=0)

In [ ]:
"""
passes_df['pass_direction'] = passes_df['calculated_pass_angle'].apply(determine_pass_direction)
passes_df['dir_zone'] = passes_df['pass_direction'].astype(str) + '_Zone_' + passes_df['Zone'].astype(str)

# Analyzing pass length
q1, q2, q3 = passes_df['calculated_pass_length'].quantile([0.33, 0.66, 1.0])
passes_df['pass_length_type'] = pd.cut(passes_df['calculated_pass_length'], bins=[0, q1, q2, q3], labels=['Short', 'Middle', 'Long'], include_lowest=True)

# Grouping, Counting, and Pivoting
grouped = passes_df.groupby(['uniqueTeamId', 'dir_zone']).size()
pivot_table = grouped.unstack(fill_value=0)
pass_length_stats = passes_df.groupby(['uniqueTeamId', 'pass_length_type']).size().unstack(fill_value=0)
average_pass_length = passes_df.groupby('uniqueTeamId')['calculated_pass_length'].mean()

# Final DataFrame
final_df = pd.concat([pivot_table, pass_length_stats, average_pass_length.rename('average_pass_length')], axis=1)

final_df.head()  # Display the top rows of the DataFrame"""


In [50]:
# Step 1: Combine 'pass_direction' and 'Zone' into a single column
passes_df['dir_zone'] = passes_df['pass_direction'].astype(str) + '_Zone_' + passes_df['Zone'].astype(str)

# Step 2: Group by 'uniqueTeamId' and the new 'dir_zone' column, then count
direction_zone_count = passes_df.groupby(['uniqueTeamId', 'dir_zone']).size()

q1, q2, q3 = passes_df['calculated_pass_length'].quantile([0.33, 0.66, 1.0])
passes_df['pass_length_type'] = pd.cut(passes_df['calculated_pass_length'], bins=[0, q1, q2, q3], labels=['Short', 'Middle', 'Long'], include_lowest=True)
pass_length_stats = passes_df.groupby(['uniqueTeamId', 'pass_length_type']).size().unstack(fill_value=0)
average_pass_length = passes_df.groupby('uniqueTeamId')['calculated_pass_length'].mean()


# Step 3: Pivot the results to have 'uniqueTeamId' as rows and each unique 'dir_zone' combination as columns
direction_zone_count_pivot = direction_zone_count.unstack(fill_value=0)

direction_count = passes_df.groupby('uniqueTeamId')['pass_direction'].value_counts().unstack(fill_value=0)

pass_direction_df = pd.concat([direction_zone_count_pivot, direction_count, pass_length_stats, average_pass_length.rename('average_pass_length')], axis=1)

<ipython-input-50-45225ee004b9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df['dir_zone'] = passes_df['pass_direction'].astype(str) + '_Zone_' + passes_df['Zone'].astype(str)


In [57]:
pass_direction_df

,Backward Pass_Zone_1,Backward Pass_Zone_2,Backward Pass_Zone_3,Forward Pass_Zone_1,Forward Pass_Zone_2,Forward Pass_Zone_3,Side Pass_Zone_1,Side Pass_Zone_2,Side Pass_Zone_3,Backward Pass,Forward Pass,Side Pass
uniqueTeamId,,,,,,,,,,,,
10531---2499726,6,38,8,57,49,6,63,76,27,52,112,166
10531---2499736,11,26,25,53,52,16,61,98,56,62,121,215
10531---2499741,6,46,16,62,78,13,69,129,19,68,153,217
10531---2499757,8,38,38,80,67,25,84,106,69,84,172,259
10531---2499766,7,25,5,51,45,3,33,34,17,37,99,84
...,...,...,...,...,...,...,...,...,...,...,...,...
756---2565880,10,30,22,51,66,24,71,100,74,62,141,245
756---2565892,10,31,13,53,64,18,54,92,48,54,135,194
756---2565904,19,30,20,75,46,12,82,57,43,69,133,182


## Pass Types

In [65]:
high_pass_types = ['High pass', 'Cross', 'Launch']
# Define low pass types based on exclusion
all_pass_types = passes_df['subEventName'].unique()
low_pass_types = [pt for pt in all_pass_types if pt not in high_pass_types and pt != 'Smart pass']

# Group by 'uniqueTeamId' and count passes
pass_counts = passes_df.groupby(['uniqueTeamId', 'subEventName']).size().unstack(fill_value=0)
pass_counts['Total_Passes'] = pass_counts.sum(axis=1)

# Calculate high pass and low pass totals
pass_counts['High_pass_Total'] = pass_counts[high_pass_types].sum(axis=1)
pass_counts['Low_pass_Total'] = pass_counts[low_pass_types].sum(axis=1)

# Calculate overall high pass and low pass ratios
pass_counts['High_pass_Ratio_Total'] = pass_counts['High_pass_Total'] / pass_counts['Total_Passes']
pass_counts['Low_pass_Ratio_Total'] = pass_counts['Low_pass_Total'] / pass_counts['Total_Passes']

# Now, we'll prepare the final DataFrame with the required columns
final_columns = ['Total_Passes', 'High_pass_Ratio_Total', 'Low_pass_Ratio_Total']
final_pass_types_df = pass_counts[final_columns]

# If needed, calculate zone-specific ratios (demonstrated for High pass, replicate for Low pass as necessary)
zones = passes_df['Zone'].unique()
for zone in zones:
    zone_specific_passes = passes_df[passes_df['Zone'] == zone]
    zone_pass_counts = zone_specific_passes.groupby(['uniqueTeamId', 'subEventName']).size().unstack(fill_value=0)
    zone_pass_counts['Total_Passes'] = zone_pass_counts.sum(axis=1)
    
    # High pass total and ratio for the zone
    zone_pass_counts['High_pass_Total_Zone'] = zone_pass_counts[high_pass_types].sum(axis=1)
    final_pass_types_df[f'High_pass_Ratio_Zone{zone}'] = zone_pass_counts['High_pass_Total_Zone'] / zone_pass_counts['Total_Passes']
    
    # Low pass total and ratio for the zone
    zone_pass_counts['Low_pass_Total_Zone'] = zone_pass_counts[low_pass_types].sum(axis=1)
    final_pass_types_df[f'Low_pass_Ratio_Zone{zone}'] = zone_pass_counts['Low_pass_Total_Zone'] / zone_pass_counts['Total_Passes']

<ipython-input-65-82b2eb0fdc0c>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_pass_types_df[f'High_pass_Ratio_Zone{zone}'] = zone_pass_counts['High_pass_Total_Zone'] / zone_pass_counts['Total_Passes']
<ipython-input-65-82b2eb0fdc0c>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_pass_types_df[f'Low_pass_Ratio_Zone{zone}'] = zone_pass_counts['Low_pass_Total_Zone'] / zone_pass_counts['Total_Passes']


In [66]:
final_pass_types_df

subEventName,Total_Passes,High_pass_Ratio_Total,Low_pass_Ratio_Total,High_pass_Ratio_Zone2,Low_pass_Ratio_Zone2,High_pass_Ratio_Zone1,Low_pass_Ratio_Zone1,High_pass_Ratio_Zone3,Low_pass_Ratio_Zone3
uniqueTeamId,,,,,,,,,
10531---2499726,330,0.127273,0.869697,0.055215,0.944785,0.198413,0.793651,0.195122,0.804878
10531---2499736,398,0.140704,0.844221,0.073864,0.909091,0.192000,0.808000,0.195876,0.773196
10531---2499741,438,0.146119,0.842466,0.130435,0.853755,0.197080,0.802920,0.083333,0.895833
10531---2499757,515,0.149515,0.829126,0.113744,0.867299,0.139535,0.860465,0.219697,0.727273
10531---2499766,220,0.240909,0.750000,0.134615,0.846154,0.351648,0.648352,0.280000,0.720000
...,...,...,...,...,...,...,...,...,...
756---2565880,448,0.214286,0.765625,0.122449,0.867347,0.204545,0.795455,0.375000,0.566667
756---2565892,383,0.177546,0.812010,0.112299,0.871658,0.136752,0.863248,0.392405,0.594937
756---2565904,384,0.270833,0.713542,0.135338,0.849624,0.301136,0.698864,0.440000,0.506667


In [25]:
merged_df = pd.concat([motif_ratios, pass_ratios], axis=1)

In [26]:
merged_df.to_csv('unique_team_id_df.csv', index=True)

In [27]:
merged_df

,ABAB,ABCA,ABCB,ABCD,High pass_Ratio,Simple pass_Ratio,Smart pass_Ratio,Cross_Ratio,Avg_Passes_per90
uniqueTeamId,,,,,,,,,
10531---2499726,0.039755,0.091743,0.088685,0.779817,0.072727,0.806061,0.003030,0.024242,3.310526
10531---2499736,0.045570,0.106329,0.103797,0.744304,0.065327,0.783920,0.015075,0.042714,3.994737
10531---2499741,0.022989,0.105747,0.105747,0.765517,0.100457,0.778539,0.011416,0.006849,4.373684
10531---2499757,0.027344,0.103516,0.101562,0.767578,0.079612,0.759223,0.021359,0.040777,5.152632
10531---2499766,0.032258,0.101382,0.105991,0.760369,0.140909,0.654545,0.009091,0.027273,2.121053
...,...,...,...,...,...,...,...,...,...
756---2565880,0.029213,0.110112,0.112360,0.748315,0.089286,0.691964,0.020089,0.093750,4.468421
756---2565892,0.036842,0.113158,0.105263,0.744737,0.093995,0.712794,0.010444,0.065274,3.794737
756---2565904,0.031496,0.089239,0.089239,0.790026,0.145833,0.601562,0.015625,0.065104,3.694737


In [145]:
final_df = pd.concat([motif_ratios, pass_df_final, direction_count, shot_df_final], axis=1)

In [146]:
final_df

,ABAB,ABCA,ABCB,ABCD,Total_Passes,High pass_Ratio,Backward Pass,Forward Pass,Side Pass,Shot,Far,Middle Distance,Near
uniqueTeamId,,,,,,,,,,,,,
10531---2499726,0.039755,0.091743,0.088685,0.779817,330,0.072727,52,112,166,4.0,1.0,1.0,2.0
10531---2499736,0.045570,0.106329,0.103797,0.744304,398,0.065327,62,121,215,7.0,3.0,4.0,0.0
10531---2499741,0.022989,0.105747,0.105747,0.765517,438,0.100457,68,153,217,6.0,1.0,2.0,3.0
10531---2499757,0.027344,0.103516,0.101562,0.767578,515,0.079612,84,172,259,8.0,0.0,5.0,3.0
10531---2499766,0.032258,0.101382,0.105991,0.760369,220,0.140909,37,99,84,3.0,0.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
756---2565880,0.029213,0.110112,0.112360,0.748315,448,0.089286,62,141,245,19.0,4.0,8.0,7.0
756---2565892,0.036842,0.113158,0.105263,0.744737,383,0.093995,54,135,194,11.0,6.0,3.0,2.0
756---2565904,0.031496,0.089239,0.089239,0.790026,384,0.145833,69,133,182,14.0,0.0,9.0,5.0


In [147]:
final_df.to_csv('final_possesion_df.csv', index=True)

# Pass_Connectivity

In [157]:
pd.DataFrame(passes_df.groupby("matchId"))

,0,1
0,2499719,eventId subEventName ...
1,2499720,eventId subEventName ...
2,2499721,eventId subEventName ...
3,2499722,eventId subEventName tags pl...
4,2499723,eventId subEventName tags pl...
...,...,...
1821,2576334,eventId subEventName tags ...
1822,2576335,eventId subEventName ...
1823,2576336,eventId subEventName tags ...
1824,2576337,eventId subEventName ...


In [156]:
passes_df

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,uniqueTeamId,calculated_pass_length,calculated_pass_angle,pass_direction
0,8,Simple pass,[{'id': 1801}],25413,"[{'y': 49, 'x': 49}, {'y': 78, 'x': 31}]",2499719,Pass,1609,1H,2.758649,85,177959171,1609---2499719,31.698580,2.320495,Side Pass
1,8,High pass,[{'id': 1801}],370224,"[{'y': 78, 'x': 31}, {'y': 75, 'x': 51}]",2499719,Pass,1609,1H,4.946850,83,177959172,1609---2499719,24.119701,-0.099669,Forward Pass
2,8,Head pass,[{'id': 1801}],3319,"[{'y': 75, 'x': 51}, {'y': 71, 'x': 35}]",2499719,Pass,1609,1H,6.542188,82,177959173,1609---2499719,19.464840,-2.976444,Backward Pass
3,8,Head pass,[{'id': 1801}],120339,"[{'y': 71, 'x': 35}, {'y': 95, 'x': 41}]",2499719,Pass,1609,1H,8.143395,82,177959174,1609---2499719,20.505609,1.212026,Side Pass
4,8,Simple pass,[{'id': 1801}],167145,"[{'y': 95, 'x': 41}, {'y': 88, 'x': 72}]",2499719,Pass,1609,1H,10.302366,85,177959175,1609---2499719,37.619144,-0.149416,Forward Pass
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3071386,8,Hand pass,[{'id': 1801}],3486,"[{'y': 29, 'x': 8}, {'y': 19, 'x': 14}]",2565927,Pass,682,2H,2925.547631,81,253302662,682---2565927,10.762899,-0.837981,Side Pass
3071388,8,Simple pass,[{'id': 1801}],3695,"[{'y': 34, 'x': 36}, {'y': 52, 'x': 45}]",2565927,Pass,682,2H,2931.782904,85,253302665,682---2565927,18.000000,0.927295,Side Pass
3071390,8,Simple pass,[{'id': 1801}],20623,"[{'y': 25, 'x': 66}, {'y': 2, 'x': 88}]",2565927,Pass,682,2H,2939.077491,85,253302671,682---2565927,32.179497,-0.608689,Forward Pass
3071392,8,Cross,"[{'id': 401}, {'id': 1802}]",122832,"[{'y': 21, 'x': 97}, {'y': 26, 'x': 92}]",2565927,Pass,682,2H,2942.098761,80,253302674,682---2565927,7.211103,2.553590,Backward Pass


In [150]:
from collections import defaultdict
import networkx as nx

In [160]:
def calculate_pass_connectivity(group):
    # Initialize two empty graphs for the two teams in the match
    team_graphs = {team_id: nx.Graph() for team_id in group['uniqueTeamId'].unique()}
    
    # Track the previous player and team to identify consecutive passes
    prev_player = None
    prev_team = None
    
    for i, pass_ in group.iterrows():
        current_player = pass_['playerId']
        current_team = pass_['uniqueTeamId']
        
        # Check for consecutive passes within the same team
        if prev_team == current_team and prev_player is not None:
            team_graphs[current_team].add_edge(prev_player, current_player)
        
        # Update the previous player and team for the next iteration
        prev_player = current_player
        prev_team = current_team
    
    # Calculate connectivity for each team graph
    connectivities = {}
    for team_id, graph in team_graphs.items():
        if len(graph) == 0:  # Handle cases where a team made no passes or only incomplete passes
            connectivity = 0
        else:
            connectivity = nx.average_node_connectivity(graph)
        connectivities[team_id] = connectivity
    
    return connectivities

In [175]:
test_dfdf = passes_df.iloc[:1000]

In [178]:
connectivity_results = [calculate_pass_connectivity(group) for _, group in passes_df.groupby(['matchId'])]

In [179]:
connectivity_results

[{'1609---2499719': 8.043956043956044, '1631---2499719': 5.989010989010989},
 {'1625---2499720': 8.252747252747254, '1651---2499720': 4.56043956043956},
 {'1646---2499721': 6.3076923076923075, '1610---2499721': 7.269230769230769},
 {'1628---2499722': 8.551282051282051, '1673---2499722': 5.736263736263736},
 {'1623---2499723': 8.483516483516484, '1639---2499723': 6.769230769230769},
 {'1611---2499724': 8.175824175824175, '1633---2499724': 7.318681318681318},
 {'1613---2499725': 6.450549450549451, '1624---2499725': 8.274725274725276},
 {'10531---2499726': 6.294871794871795, '1619---2499726': 7.758241758241758},
 {'1644---2499727': 6.846153846153846, '1612---2499727': 5.736263736263736},
 {'1627---2499728': 6.256410256410256, '1659---2499728': 6.725274725274725},
 {'1644---2499729': 5.967032967032967, '1659---2499729': 6.582417582417582},
 {'1627---2499730': 4.589743589743589, '1646---2499730': 7.217948717948718},
 {'1673---2499731': 7.0256410256410255, '1613---2499731': 7.362637362637362

In [185]:
connectivity_df.set_index('uniqueTeamId', inplace=True)
connectivity_df

,Connectivity
uniqueTeamId,
1609---2499719,8.043956
1631---2499719,5.989011
1625---2499720,8.252747
1651---2499720,4.560440
1646---2499721,6.307692
...,...
3158---2576336,8.285714
3164---2576337,8.333333
3204---2576337,8.410256


In [182]:
connectivity_df = [(k, v) for d in connectivity_results for k, v in d.items()]
connectivity_df = pd.DataFrame(connectivity_df, columns=['uniqueTeamId', 'Connectivity'])

In [186]:
fin_df = pd.concat([motif_ratios, pass_df_final, pass_direction_df, shot_df_final, connectivity_df], axis=1)

In [188]:
fin_df.to_csv('harbi_final_possesion_df.csv', index=True)

In [97]:
final_df = pd.concat([motif_ratios, final_pass_types_df, pass_direction_df, shot_df_final, connectivity_df], axis=1)
# Iterate through each column in final_df to find and adjust the specified columns
for col in final_df.columns:
    # Check if the column name contains 'Backward', 'Forward', or 'Side'
    if any(keyword in col for keyword in ['Backward', 'Forward', 'Side']):
        # Divide the column by the 'Total_Passes' column
        final_df[col] = final_df[col] / final_df['Total_Passes']
        # Rename the column to add " Ratio" at the end
        final_df.rename(columns={col: col + " Ratio"}, inplace=True)

In [98]:
final_df

,ABAB,ABCA,ABCB,ABCD,Total_Passes,High_pass_Ratio_Total,Low_pass_Ratio_Total,High_pass_Ratio_Zone2,Low_pass_Ratio_Zone2,High_pass_Ratio_Zone1,...,Side Pass_Zone_2 Ratio,Side Pass_Zone_3 Ratio,Backward Pass Ratio,Forward Pass Ratio,Side Pass Ratio,Total Shots,Far Shots Ratio,Middle Distance Shots Ratio,Near Shots Ratio,Connectivity
uniqueTeamId,,,,,,,,,,,,,,,,,,,,,
10531---2499726,0.039755,0.091743,0.088685,0.779817,330,0.127273,0.869697,0.055215,0.944785,0.198413,...,0.230303,0.081818,0.157576,0.339394,0.503030,4.0,0.250000,0.250000,0.500000,6.294872
10531---2499736,0.045570,0.106329,0.103797,0.744304,398,0.140704,0.844221,0.073864,0.909091,0.192000,...,0.246231,0.140704,0.155779,0.304020,0.540201,7.0,0.428571,0.571429,0.000000,6.659341
10531---2499741,0.022989,0.105747,0.105747,0.765517,438,0.146119,0.842466,0.130435,0.853755,0.197080,...,0.294521,0.043379,0.155251,0.349315,0.495434,6.0,0.166667,0.333333,0.500000,6.472527
10531---2499757,0.027344,0.103516,0.101562,0.767578,515,0.149515,0.829126,0.113744,0.867299,0.139535,...,0.205825,0.133981,0.163107,0.333981,0.502913,8.0,0.000000,0.625000,0.375000,7.153846
10531---2499766,0.032258,0.101382,0.105991,0.760369,220,0.240909,0.750000,0.134615,0.846154,0.351648,...,0.154545,0.077273,0.168182,0.450000,0.381818,3.0,0.000000,0.666667,0.333333,5.285714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
756---2565880,0.029213,0.110112,0.112360,0.748315,448,0.214286,0.765625,0.122449,0.867347,0.204545,...,0.223214,0.165179,0.138393,0.314732,0.546875,19.0,0.210526,0.421053,0.368421,6.615385
756---2565892,0.036842,0.113158,0.105263,0.744737,383,0.177546,0.812010,0.112299,0.871658,0.136752,...,0.240209,0.125326,0.140992,0.352480,0.506527,11.0,0.545455,0.272727,0.181818,7.362637
756---2565904,0.031496,0.089239,0.089239,0.790026,384,0.270833,0.713542,0.135338,0.849624,0.301136,...,0.148438,0.111979,0.179688,0.346354,0.473958,14.0,0.000000,0.642857,0.357143,8.089744


In [99]:
final_df.to_csv('final_final_possesion_df.csv', index=True)

# END OF FILE

In [103]:
np.unique(all_events_df.subEventName)

array(['', 'Acceleration', 'Air duel', 'Ball out of the field',
       'Clearance', 'Corner', 'Cross', 'Foul', 'Free Kick',
       'Free kick cross', 'Free kick shot', 'Goal kick',
       'Goalkeeper leaving line', 'Ground attacking duel',
       'Ground defending duel', 'Ground loose ball duel', 'Hand foul',
       'Hand pass', 'Head pass', 'High pass', 'Late card foul', 'Launch',
       'Out of game foul', 'Penalty', 'Protest', 'Reflexes',
       'Save attempt', 'Shot', 'Simple pass', 'Simulation', 'Smart pass',
       'Throw in', 'Time lost foul', 'Touch', 'Violent Foul', 'Whistle'],
      dtype=object)